<br>

<div align=center><font color=maroon size=6 style="line-height:40px;"><b>Introduction to tensor slicing</b></font></div>

<br>

<font size=4><b>References:</b></font>
1. TF2 Core: <a href="https://www.tensorflow.org/guide" style="text-decoration:none;">TensorFlow Guide</a> 
    * `TensorFlow > Learn > TensorFlow Core > `Guide > <a href="https://www.tensorflow.org/guide/tensor_slicing" style="text-decoration:none;">Introduction to tensor slicing</a>
        * Run in <a href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/tensor_slicing.ipynb" style="text-decoration:none;">Google Colab</a>

<br>
<br>
<br>

When working on ML applications such as object detection and NLP, it is sometimes necessary to work with sub-sections (slices) of tensors. For example, if your model architecture includes routing, where one layer might control which training example gets routed to the next layer. In this case, you could use tensor slicing ops to split the tensors up and put them back together in the right order.

In NLP applications, you can use tensor slicing to perform word masking while training. For example, you can generate training data from a list of sentences by choosing a word index to mask in each sentence, taking the word out as a label, and then replacing the chosen word with a mask token.  

In this guide, you will learn how to use the TensorFlow APIs to:

* Extract slices from a tensor
* Insert data at specific indices in a tensor

This guide assumes familiarity with tensor indexing. Read the indexing sections of the [Tensor](https://www.tensorflow.org/guide/tensor#indexing) and [TensorFlow NumPy](https://www.tensorflow.org/guide/tf_numpy#indexing) guides before getting started with this guide.

<br>
<br>

In [1]:
import tensorflow as tf

import numpy as np

In [2]:
print(tf.__version__)

2.8.0


<br>

## Extract tensor slices

<font style="color:maroon;font-size:115%">Perform NumPy-like tensor slicing using `tf.slice`.</font>

In [3]:
t1 = tf.constant([0, 1, 2, 3, 4, 5, 6, 7])

print(tf.slice(t1, begin=[1], size=[3]))

tf.Tensor([1 2 3], shape=(3,), dtype=int32)


<br>

<font style="color:maroon;font-size:115%">Alternatively, you can use a more Pythonic syntax. Note that tensor slices are evenly spaced over a start-stop range.</font>

In [4]:
print(t1[1:4])

tf.Tensor([1 2 3], shape=(3,), dtype=int32)


In [5]:
print(t1[-3:])

tf.Tensor([5 6 7], shape=(3,), dtype=int32)


<br>

<font style="color:maroon;font-size:115%">For 2-dimensional tensors,you can use something like:</font>

In [6]:
t2 = tf.constant([[0, 1, 2, 3, 4],
                  [5, 6, 7, 8, 9],
                  [10, 11, 12, 13, 14],
                  [15, 16, 17, 18, 19]])

print(t2[:-1, 1:3])

tf.Tensor(
[[ 1  2]
 [ 6  7]
 [11 12]], shape=(3, 2), dtype=int32)


<br>

<font style="color:maroon;font-size:115%">You can use `tf.slice` on higher dimensional tensors as well.</font>

In [8]:
t3 = tf.constant([[[1, 3, 5, 7],
                   [9, 11, 13, 15]],
                  
                  [[17, 19, 21, 23],
                   [25, 27, 29, 31]]
                  ])  # (2, 2, 4)
print(t3.shape, '\n')

print(tf.slice(t3, begin=[1, 1, 0], size=[1, 1, 2]))

(2, 2, 4) 

tf.Tensor([[[25 27]]], shape=(1, 1, 2), dtype=int32)


<br>

<font style="color:maroon;font-size:120%">You can also use `tf.strided_slice` to extract slices of tensors by 'striding' over the tensor dimensions.</font>

<br>

<font style="color:maroon;font-size:120%">Use `tf.gather` to extract specific indices from a single axis of a tensor.</font>

In [9]:
t1

<tf.Tensor: shape=(8,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7])>

In [10]:
print(tf.gather(t1, indices=[0, 3, 6]))

# This is similar to doing

t1[::3]

tf.Tensor([0 3 6], shape=(3,), dtype=int32)


<tf.Tensor: shape=(3,), dtype=int32, numpy=array([0, 3, 6])>

<br>

<font style="color:maroon;font-size:120%">`tf.gather` does not require indices to be evenly spaced.</font>

In [11]:
alphabet = tf.constant(list('abcdefghijklmnopqrstuvwxyz'))

print(tf.gather(alphabet, indices=[2, 0, 19, 18]))

tf.Tensor([b'c' b'a' b't' b's'], shape=(4,), dtype=string)


<br>

<img src="images/gather_1.png" width=500px>

<br>

<font style="color:maroon;font-size:120%">To extract slices from multiple axes of a tensor, use `tf.gather_nd`. This is useful when you want to gather the elements of a matrix as opposed to just its rows or columns.</font>

In [12]:
t4 = tf.constant([[0, 5],
                  [1, 6],
                  [2, 7],
                  [3, 8],
                  [4, 9]])

print(tf.gather_nd(t4, indices=[[2], [3], [0]]))

tf.Tensor(
[[2 7]
 [3 8]
 [0 5]], shape=(3, 2), dtype=int32)


<br>

<img src="images/gather_2.png" width=500px>

<br>
<br>

In [19]:
t5 = np.reshape(np.arange(18), [2, 3, 3])
print(t5.shape, '\n')
print(t5, '\n\n')

print(tf.gather_nd(t5, indices=[[0, 0, 0], [1, 2, 1]]))
# 因为每一个 indices 的元素都是 `三维的`， t5 也是三维的。
# 故 每一个 indices 的元素对应了 t5 的一个坐标
# t5 中坐标为（0,0,0) 和 (1,2,1) 的两个元素。

(2, 3, 3) 

[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]] 


tf.Tensor([ 0 16], shape=(2,), dtype=int32)


<br>

In [17]:
# Return a list of two matrices

print(tf.gather_nd(t5,    # shape=(2,3,3)
                   indices=[ [[0, 0], [0, 2]],
                             [[1, 0], [1, 2]]
                           ]))

# 因为每一个 indices 的每个元素都是 一个 2x2 的矩阵。该矩阵是 2 维矩阵；该矩阵的每个元素（每一行）又包括了 2 个元素。
# 而 t5 是三维的。shape=(2,3,3)
# 因此 indices 的每个元素（2x2的矩阵）的每一行（含有2个元素）对应了 t5 的三个维度的前两个维度。
# 故 [[0,0],  [0,2]] 表示 t5[0, 0, :] 和 t5[0, 2, :]。
#   [[1,0],  [1,2]] 表示 t5[1, 0, :] 和 t5[1, 2, :]。

tf.Tensor(
[[[ 0  1  2]
  [ 6  7  8]]

 [[ 9 10 11]
  [15 16 17]]], shape=(2, 2, 3), dtype=int32)


<br>

In [20]:
print(t5[0, 0, :], "\n")
print(t5[0, 2, :], "\n")

[0 1 2] 

[6 7 8] 



<br>
<br>

In [18]:
# Return one matrix

print(tf.gather_nd(t5,
                   indices=[[0, 0], [0, 2], [1, 0], [1, 2]]))

tf.Tensor(
[[ 0  1  2]
 [ 6  7  8]
 [ 9 10 11]
 [15 16 17]], shape=(4, 3), dtype=int32)


<br>
<br>
<br>

## Insert data into tensors

<font style="color:maroon;font-size:115%">Use `tf.scatter_nd` to insert data at specific slices/indices of a tensor.<br><br>

**Note that** the tensor into which you insert values is zero-initialized.</font>

In [22]:
t6 = tf.constant([10])
print(t6, "\n")

indices = tf.constant([[1], [3], [5], [7], [9]])
data = tf.constant([2, 4, 6, 8, 10])   # indices 处插入的值

print(tf.scatter_nd(indices=indices,
                    updates=data,
                    shape=t6))

tf.Tensor([10], shape=(1,), dtype=int32) 

tf.Tensor([ 0  2  0  4  0  6  0  8  0 10], shape=(10,), dtype=int32)


<br>
<br>

<font style="color:maroon;font-size:120%">Methods like `tf.scatter_nd` which require zero-initialized tensors are similar to sparse tensor initializers. You can use `tf.gather_nd` and `tf.scatter_nd` to mimic the behavior of sparse tensor ops.</font>

Consider an example where you construct a sparse tensor using these two methods in conjunction.

In [24]:
# Gather values from one tensor by specifying indices
print(t2, "\n\n")

new_indices = tf.constant([[0, 2], [2, 1], [3, 3]])

t7 = tf.gather_nd(t2, indices=new_indices)
print(t7)

tf.Tensor(
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]], shape=(4, 5), dtype=int32) 


tf.Tensor([ 2 11 18], shape=(3,), dtype=int32)


<br>

<img src="images/gather_nd_sparse.png" width=500px>

<br>

In [25]:
# Add these values into a new tensor

t8 = tf.scatter_nd(indices=new_indices, updates=t7, shape=tf.constant([4, 5]))

print(t8)

tf.Tensor(
[[ 0  0  2  0  0]
 [ 0  0  0  0  0]
 [ 0 11  0  0  0]
 [ 0  0  0 18  0]], shape=(4, 5), dtype=int32)


<br>

<font style="color:maroon;font-size:115%">This is similar to:</font>

In [26]:
t9 = tf.SparseTensor(indices=[[0, 2], [2, 1], [3, 3]],
                     values=[2, 11, 18],
                     dense_shape=[4, 5])

print(t9)

SparseTensor(indices=tf.Tensor(
[[0 2]
 [2 1]
 [3 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([ 2 11 18], shape=(3,), dtype=int32), dense_shape=tf.Tensor([4 5], shape=(2,), dtype=int64))


In [27]:
# Convert the sparse tensor into a dense tensor

t10 = tf.sparse.to_dense(t9)

print(t10)

tf.Tensor(
[[ 0  0  2  0  0]
 [ 0  0  0  0  0]
 [ 0 11  0  0  0]
 [ 0  0  0 18  0]], shape=(4, 5), dtype=int32)


<br>

<font style="color:maroon;font-size:120%">To insert data into a tensor with pre-existing values, use `tf.tensor_scatter_nd_add`.</font>

In [37]:
t11 = tf.constant([[2, 7, 0],
                   [9, 0, 1],
                   [0, 3, 8]])

# Convert the tensor into a magic square by inserting numbers at appropriate indices

t12 = tf.tensor_scatter_nd_add(t11,
                               indices=[[0, 2], [1, 1], [2, 0]],
                               updates=[6, 5, 4])

print(t12)

tf.Tensor(
[[2 7 6]
 [9 5 1]
 [4 3 8]], shape=(3, 3), dtype=int32)


<br>

<font style="color:maroon;font-size:115%">Similarly, use `tf.tensor_scatter_nd_sub` to subtract values from a tensor with pre-existing values.</font>

In [38]:
# Convert the tensor into an identity matrix

t13 = tf.tensor_scatter_nd_sub(t11,
                               indices=[[0, 0], [0, 1], [1, 0], [1, 1], [1, 2], [2, 1], [2, 2]],
                               updates=[1, 7, 9, -1, 1, 3, 7])

print(t13)

tf.Tensor(
[[1 0 0]
 [0 1 0]
 [0 0 1]], shape=(3, 3), dtype=int32)


<br>

<font style="color:maroon;font-size:115%">Use `tf.tensor_scatter_nd_min` to copy element-wise minimum values from one tensor to another.</font>

In [39]:
t14 = tf.constant([[-2, -7, 0],
                   [-9, 0, 1],
                   [0, -3, -8]])

t15 = tf.tensor_scatter_nd_min(t14,
                               indices=[[0, 2], [1, 1], [2, 0]],
                               updates=[-6, -5, -4])

print(t15)

tf.Tensor(
[[-2 -7 -6]
 [-9 -5  1]
 [-4 -3 -8]], shape=(3, 3), dtype=int32)


<br>

In [40]:
t14 = tf.constant([[-2, -7, 10],
                   [-9, 10, 1],
                   [10, -3, -8]])

t15 = tf.tensor_scatter_nd_min(t14,
                               indices=[[0, 2], [1, 1], [2, 0]],
                               updates=[-6, -5, -4])

print(t15)

tf.Tensor(
[[-2 -7 -6]
 [-9 -5  1]
 [-4 -3 -8]], shape=(3, 3), dtype=int32)


<br>

<font style="color:maroon;font-size:115%">Similarly, use `tf.tensor_scatter_nd_max` to copy element-wise maximum values from one tensor to another.</font>

In [41]:
t14 = tf.constant([[-2, -7, 0],
                   [-9, 0, 1],
                   [0, -3, -8]])

t16 = tf.tensor_scatter_nd_max(t14,
                               indices=[[0, 2], [1, 1], [2, 0]],
                               updates=[6, 5, 4])

print(t16)

tf.Tensor(
[[-2 -7  6]
 [-9  5  1]
 [ 4 -3 -8]], shape=(3, 3), dtype=int32)


<br>
<br>
<br>

## Further reading and resources

In this guide, you learned how to use the tensor slicing ops available with TensorFlow to exert finer control over the elements in your tensors.

* Check out the slicing ops available with TensorFlow NumPy such as `tf.experimental.numpy.take_along_axis` and `tf.experimental.numpy.take`.

* Also check out the [Tensor guide](https://www.tensorflow.org/guide/tensor) and the [Variable guide](https://www.tensorflow.org/guide/variable).

<br>
<br>
<br>